In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, convert_to_messages
from os.path import dirname, join
from dotenv import load_dotenv
from typing import Literal
from typing_extensions import TypedDict, Annotated
from langgraph.graph import StateGraph, START, MessagesState, END
from langgraph.types import Command
from operator import add
import pandas as pd
import getpass
import os

In [ ]:
# Set your OpenAI API key
load_dotenv("/Users/mastorga/Documents/BTE-LLM/.env")

if not os.environ.get("OPENAI_API_KEY"): #field to ask for OpenAI API key
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter OpenAI API Key: ")

In [ ]:
from Agent import BTEx

In [ ]:
# Create an LLM-based agent
llm = ChatOpenAI(temperature=0, model="gpt-4.1")  # Change model if needed

In [ ]:
def get_llm_response(question):
    return llm.invoke(question).content

In [ ]:
drugDiseaseDMDB = pd.read_csv('/Users/mastorga/Documents/BTE-LLM/Prototype/data/DMDB_chebi_metabolite_filtered.csv')

In [ ]:
drugDiseaseSet = drugDiseaseDMDB.sample(25)

print(drugDiseaseSet)

In [ ]:
# LLM answer (w/o ID)

drugDiseaseSet['gpt4.1_response w/o ID'] = drugDiseaseSet["question"].apply(get_llm_response)

In [ ]:

drugDiseaseSet.to_excel('/Users/mastorga/Documents/BTE-LLM/Prototype/logs/drug <- diseasebp/08.20.25_metabolites_25q-gpt4.1.xlsx')

In [ ]:
# BTE-RAG (maxresults: 100, k = 5) (w/o ID)

drugDiseaseSet["BTEx_response (w/o ID) (maxresult = 100, k = 5)"] = drugDiseaseSet["question"].apply(lambda q: BTEx(q, maxresults=100, k=5))

In [ ]:
# Save responses

drugDiseaseSet.to_excel('/Users/mastorga/Documents/BTE-LLM/Prototype/logs/drug <- diseasebp/RAG_Response_08.20.25_metabolites_25q-gpt4.1.xlsx')

In [ ]:
# list of response columns you want to check
response_cols = [
    "gpt4.1_response w/o ID",
    "BTEx_response (w/o ID) (maxresult = 100, k = 5)"
]

# for each response column, create a new boolean column
for col in response_cols:
    new_col = f"contains_drug_{col}"
    drugDiseaseSet[new_col] = drugDiseaseSet.apply(
        lambda row: f"{str(row['metabolite_name_str']).lower()}" in str(row[col]).lower(),
        axis=1
    )

In [ ]:
# Save responses

drugDiseaseSet.to_excel('/Users/mastorga/Documents/BTE-LLM/Prototype/logs/drug <- diseasebp/checkedRAG_Response_08.20.25_metabolites_25q-gpt4.1.xlsx')